# 데이터 수집

 - 첫번째 파이프 라인은 '데이터 수집'입니다. 대회측 데이터에 대한 train_test_split과 저희 팀이 수집한 4가지 데이터들에 대해서 수집방법, 전처리방법, 그리고 데이터 통합까지 담았습니다.  


     - 0. 대회측 제공 데이터
     - 1. 기상정보 데이터
     - 2. 수위, 유량 데이터
     - 3. 다목적댐 관리현황 데이터
     - 4. 수문 및 기타 데이터
     - 5. 데이터 통합
     
***

In [1]:
# Data Handling
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')
import arrow
import joblib
import os
pd.set_option('max_columns', 30)

# Data Crawling
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote_plus

### 0. 대회측 제공 데이터

In [6]:
data_name = '2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803'
column_names = [
    '홍수사상번호', '연', '월', '일', '시간', '유입량',
    '유역평균강수','강우(A지역)','강우(B지역)','강우(C지역)','강우(D지역)','수위(E지역)','수위(D지역)', # 데이터집단1
    '유역평균강수','강우(A지역)','강우(B지역)','강우(C지역)','강우(D지역)','수위(E지역)','수위(D지역)', # 데이터집단2
    '유역평균강수','강우(A지역)','강우(B지역)','강우(C지역)','강우(D지역)','수위(E지역)','수위(D지역)', # 데이터집단3
    '유역평균강수','강우(A지역)','강우(B지역)','강우(C지역)','강우(D지역)','수위(E지역)','수위(D지역)', # 데이터집단4
    '유역평균강수','강우(A지역)','강우(B지역)','강우(C지역)','강우(D지역)','수위(E지역)','수위(D지역)', # 데이터집단5
    '유역평균강수','강우(A지역)','강우(B지역)','강우(C지역)','강우(D지역)','수위(E지역)','수위(D지역)', # 데이터집단6
]

# 데이터 로드, 데이터 스플릿
data = pd.read_excel(f'./data/{data_name}.xlsx', header=1, names=column_names)
test = data.iloc[2891:]; test.drop('유입량', axis=1, inplace=True)
train = data.iloc[:2891]
target = train['유입량']; train.drop('유입량', axis=1, inplace=True)


# 데이터 형태 확인
print(f'train shape: {train.shape}   \
      \ntarget shape: {target.shape} \
      \ntest shape: {test.shape}')


# 데이터 저장
data.to_csv('./data/data.csv', index=False)
train.to_csv('./data/train.csv', index=False)
target.to_csv('./data/target.csv', index=False)
test.to_csv('./data/test.csv', index=False)

train shape: (2891, 47)         
target shape: (2891,)       
test shape: (160, 47)


### 1. 기상정보 데이터 수집
 - 기상자료개방포털의 https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36 에서 연도별 csv파일을 다운받아 수집하였음

 - Data Load

In [3]:
# 수집한 기상 데이터 로드
filelst = os.listdir('./data/기상정보데이터연도별')
data_lst = [pd.read_csv(f'./data/기상정보데이터연도별/{file}', encoding='cp949') for file in filelst]
w_df = pd.concat(data_lst)


# 지점 열 삭제 
w_df = w_df.iloc[:,1:]
w_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804742 entries, 0 to 69476
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   지점명          804742 non-null  object 
 1   일시           804742 non-null  object 
 2   기온(°C)       804708 non-null  float64
 3   풍속(m/s)      804523 non-null  float64
 4   습도(%)        804415 non-null  float64
 5   증기압(hPa)     742139 non-null  float64
 6   이슬점온도(°C)    741975 non-null  float64
 7   현지기압(hPa)    741814 non-null  float64
 8   해면기압(hPa)    742262 non-null  float64
 9   전운량(10분위)    170356 non-null  float64
 10  중하층운량(10분위)  170259 non-null  float64
 11  최저운고(100m )  112143 non-null  float64
 12  지면온도(°C)     726158 non-null  float64
dtypes: float64(11), object(2)
memory usage: 86.0+ MB


 - Missing Data 처리

In [4]:
# 전운량, 중하층운량,최저운고는 결측치가 너무 많으므로 삭제
# 현지기압은 해면기압만 사용해도 충분하다 판단하여 삭제 
w_df.drop(['전운량(10분위)','중하층운량(10분위)','최저운고(100m )'],axis=1,inplace=True)
w_df.drop(['현지기압(hPa)'],axis=1,inplace=True)


# 인덱스 정리 
w_df = w_df.reset_index()
w_df.drop(['index'],axis=1,inplace=True)


# nan값 처리를 위해서 열별로 nan값이 들어있는 행의 인덱스를 추출
temp_nan_index = np.where(w_df['기온(°C)'].isna())[0].tolist()      # 기온 열의 결측값이 있는 인덱스 리스트 
wind_nan_index = np.where(w_df['풍속(m/s)'].isna())[0].tolist()      # 풍속 결측값 인덱스 
hum_nan_index = np.where(w_df['습도(%)'].isna())[0].tolist()         # 습도 결측값 인덱스 
pha_nan_index = np.where(w_df['증기압(hPa)'].isna())[0].tolist()     # 증기압 결측값 인덱스 
dew_nan_index = np.where(w_df['이슬점온도(°C)'].isna())[0].tolist() # 이슬점 결측값 인덱스
sea_nan_index = np.where(w_df['해면기압(hPa)'].isna())[0].tolist()   # 해면기압 결측값 인덱스
ltemp_nan_index = np.where(w_df['지면온도(°C)'].isna())[0].tolist() # 지면온도 결측값 인덱스 


# 결측값 처리 함수 : 해당 결측값의 위, 아래 시간대의 평균값으로 nan값 대체 
def Missing_value(column,index_list):
    for nan in index_list:
        w_df.loc[(w_df.index == nan),column] = w_df.loc[[nan-1,nan+1]][column].mean()
# 결측값 처리 함수 적용      
Missing_value('기온(°C)',temp_nan_index)
Missing_value('풍속(m/s)',wind_nan_index)
Missing_value('습도(%)',hum_nan_index)
Missing_value('증기압(hPa)',pha_nan_index)
Missing_value('이슬점온도(°C)',dew_nan_index)
Missing_value('해면기압(hPa)',sea_nan_index)


# 지면온도 결측값 처리 : 해당 날의 평균 온도
# 지면온도 결측값 처리를 위하여 시간을 제외한'일' 열 생성 
w_df['일'] = w_df['일시'].apply(lambda x : x[8:10])
for nan in ltemp_nan_index :
    date = w_df.loc[nan]['일']
    same_day = w_df.query('일 == @date')
    temp_mean = round(same_day['지면온도(°C)'].mean(),1)
    w_df.loc[(w_df.index == nan),'지면온도(°C)'] = temp_mean
    
    
w_df.isna().sum()

지점명          0
일시           0
기온(°C)       0
풍속(m/s)      0
습도(%)        0
증기압(hPa)     0
이슬점온도(°C)    0
해면기압(hPa)    0
지면온도(°C)     0
일            0
dtype: int64

 - Data Processing

In [53]:
# 생성했던 '일'열 제거 
w_df.drop(['일'],axis=1,inplace=True)


# weather 정의
weather = w_df.groupby('일시').mean().reset_index()


# 원데이터와의 merge를 위한 기준열 생성 
weather['연'] = weather['일시'].apply(lambda x : int(x[:4]))
weather['월'] = weather['일시'].apply(lambda x : int(x[6:7]))
weather['일'] = weather['일시'].apply(lambda x : int(x[8:10]))
weather['시간'] = weather['일시'].apply(lambda x : int(x[11:13]))

weather.drop(['일시'],axis=1,inplace=True)
weather.head()


# train데이터에 붙이기 위해 train 데이터형태로 만들기
# 0시를 24시로 바꾼다. 
weather['시간'] = weather['시간'].apply(lambda x : 24 if (x == 0) else x )


# 원본데이터와 같게 만들기 위해 18번 인덱스의 날짜를 변경 
weather.loc[(weather.index == 18),'일'] = 10
standard_d = data[['연','월','일','시간']]


# nan 값이 뜨는 40번 행에 대해서 39번과 41번의 평균을 내려서 데이터를 기입함
merge_df.loc[40] = (merge_df.iloc[39] + merge_df.iloc[41]) / 2
# 칼럼 rename
new_name = ['기온', '풍속', '습도', '증기압', '이슬점온도', '해면기압', '지면온도']
merge_df = merge_df.rename(columns=dict(zip(merge_df.columns, new_name)))


# 최종 데이터 저장 
merge_df.to_csv('./data/기상데이터_최종.csv',encoding='cp949',index=False)
# 기상 데이터 확인 
pd.read_csv('./data/기상데이터_최종.csv',encoding='cp949')

,기온(°C),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),해면기압(hPa),지면온도(°C)
0,22.314286,2.200000,84.857143,23.078571,19.657143,1007.864286,12.700000
1,22.385714,2.200000,85.428571,23.500000,19.942857,1007.271429,22.242857
2,21.642857,2.871429,86.428571,23.500000,19.942857,1007.271429,12.700000
3,21.128571,2.542857,87.857143,22.642857,19.328571,1006.385714,12.700000
4,21.128571,2.900000,85.714286,21.785714,18.714286,1005.500000,12.700000
...,...,...,...,...,...,...,...
3046,20.937500,3.162500,59.125000,14.025000,11.950000,1011.200000,27.112500
3047,19.975000,3.037500,61.500000,13.675000,11.562500,1011.737500,24.037500
3048,18.912500,2.425000,63.500000,13.337500,11.175000,1012.362500,21.550000
3049,18.012500,2.187500,66.375000,13.262500,11.100000,1012.962500,19.925000


### 2. 수위관측소(수위, 유량) 데이터 수집
 - K-water에서 관리중인 수위관측소 데이터를
 http://opendata.kwater.or.kr/pubdata/dam/exllncobsrvt.do?seq_no=40 에서 연도별 엑셀 파일들을 다운받아 사용하였음

In [132]:
# 폴더에 있는 파일들을 불러온다
filelst = os.listdir('./data/수위관측소지역별연도별')


# 지역명을 넣으면 해당 지역의 2006~2018년의 데이터를 통합하여 리턴하는 함수
def return_observatory_data(region_name):
    region_data_lst = [file for file in filelst if file.startswith(f'{region_name}')]
    df_lst = []
    for i in range(len(region_data_lst)):
        df_lst.append(pd.read_csv(f'./data/수위관측소지역별연도별/{region_data_lst[i]}', encoding='cp949'))
    df = pd.concat(df_lst, ignore_index=True).iloc[:, 2:]
    time = pd.concat(df_lst, ignore_index=True).iloc[:, 1]
    df.columns = [f'{region_name}_수위',f'{region_name}_유량']
    return df, time


# 함수를 적용하여 지역별 데이터를 정의하고 하나로 데이터를 합함
Osa_ri_df, time = return_observatory_data('단양군(오사리)')
Bukssang_ri_df, _ = return_observatory_data('영월군(북쌍리)')
Panun_bridge_df, _ = return_observatory_data('영월군(판운교)')
Chungju_dam_df, _ = return_observatory_data('우안(충주댐)')
Cheongpunggyo_bridge_df, _ = return_observatory_data('제천시(청풍교)')
Myungseogyo_bridge_df, _ = return_observatory_data('충주시(명서교)')
Hyangsan_ri_df, _ = return_observatory_data('충주시(향산리)')
dfs = pd.concat([time, Osa_ri_df, Bukssang_ri_df, Panun_bridge_df, Chungju_dam_df, Cheongpunggyo_bridge_df, Myungseogyo_bridge_df, Hyangsan_ri_df], axis=1)


# 차후 통합을 위해 날짜 칼럼을 분할함
dfs['시간'] = dfs['시간'].apply(lambda x: x[:13])
dfs['연'] = dfs['시간'].apply(lambda x : x[:4]).astype(int)
dfs['월'] = dfs['시간'].apply(lambda x : x[6:7]).astype(int)
dfs['일'] = dfs['시간'].apply(lambda x : x[8:10]).astype(int)
dfs['시간'] = dfs['시간'].apply(lambda x : x[11:13]).astype(int)


# 데이터의 형태 조정 및 저장
observatory_df = pd.merge(data, dfs, on=['연', '월', '일', '시간'], how='left').iloc[:, 48:]


# 일부 nan값 수정
observatory_df.loc[1532, :] = (observatory_df.loc[1531, :] + observatory_df.loc[1533, :]) / 2
observatory_df.loc[2760, :] = (observatory_df.loc[2759, :] + observatory_df.loc[2761, :]) / 2
# 제천시(청풍교)와 우안(충주댐)의 데이터가 전부 0이어서 해당 칼럼 드랍
observatory_df.drop(['제천시(청풍교)_수위', '충주시(명서교)_수위', '우안(충주댐)_수위'], axis=1, inplace=True)


# 데이터 저장 및 확인
observatory_df.to_excel('./data/수위관측소(수위,유량).xlsx',index=False)
pd.read_excel('./data/수위관측소(수위,유량).xlsx')


# 데이터 저장 및 확인
observatory_df.to_excel('./data/수위관측소(수위,유량).xlsx',index=False)
pd.read_excel('./data/수위관측소(수위,유량).xlsx')

,단양군(오사리)_수위,단양군(오사리)_유량,영월군(북쌍리)_수위,영월군(북쌍리)_유량,영월군(판운교)_수위,영월군(판운교)_유량,우안(충주댐)_유량,제천시(청풍교)_유량,충주시(명서교)_유량,충주시(향산리)_수위,충주시(향산리)_유량
0,147.50,2.54,61.50,0.96,23.60,1.51,122.65,122.63,0.99,24.00,0.37
1,145.80,2.53,61.50,0.96,23.60,1.51,122.63,122.62,1.00,24.00,0.37
2,145.80,2.53,61.50,0.96,23.60,1.51,122.59,122.59,1.01,24.00,0.37
3,145.80,2.53,61.50,0.96,23.60,1.51,122.56,122.55,1.01,24.00,0.37
4,145.80,2.53,61.50,0.96,23.60,1.51,122.54,122.51,1.01,24.00,0.37
...,...,...,...,...,...,...,...,...,...,...,...
3046,455.90,3.16,78.99,1.10,52.66,1.62,129.92,130.05,1.15,21.12,0.36
3047,451.46,3.15,77.63,1.09,52.66,1.62,129.93,130.01,1.15,20.24,0.35
3048,442.67,3.13,76.28,1.08,46.39,1.54,129.91,130.03,1.15,20.24,0.35
3049,433.99,3.11,74.93,1.07,50.26,1.59,129.90,129.99,1.14,20.24,0.35


### 3. 다목적댐 관리현황 데이터 수집
 - K-water에서 관리중인 다목적댐 운영정보를 https://www.data.go.kr/data/3074799/openapi.do 에서 API를 통해서 수집하였음

In [ ]:
# API를 통해 받은 자료를 데이터프레임으로 추출해주는 함수
def api_dataframe(url, queryParams):

    res = requests.get(url + queryParams).text
    xmlobj = bs4.BeautifulSoup(res, 'lxml-xml')

    # xml로 호출한 API 데이터를 DataFrame 형태로 변환

    rows = xmlobj.findAll('item')
    rowlst = []
    columnlst = []
    namelst = []

    rowsLen = len(rows)
    for i in range(0, rowsLen):
        columns = rows[i].find_all()
        columnsLen = len(columns)

        for j in range(0, columnsLen):
            if i == 0:
                namelst.append(columns[j].name)
            eachColumn = columns[j].text
            columnlst.append(eachColumn)
        rowlst.append(columnlst)
        columnlst = []

    result = pd.DataFrame(rowlst, columns=namelst)
    
    return result

 - API 요청 변수 리스트

In [ ]:
# 요청변수 리스트
ServiceKey = 'i+FVXI3jHGn45sy7L5L5+fv4Udn73xGZN0QDoo38oM0ofEL/qtlew95sCJufGm39jEBsnQnzr0dFPxjIEz1cbw=='
vdates = [] # 선택날자
tdates = [] # 전일(선택날자 기준)
ldates = [] # 전년도(선택날자 기준)
vtimes = [] # 조회시간
numOfRows = '100'
pageNo = '1'
for i in range(len(train)):
    vdate = str(train['연'][i]) + '-'  + str(train['월'][i]) + '-' + str(train['일'][i])
    vdate = arrow.get(vdate).format('YYYY-MM-DD')
    tdate = arrow.get(vdate).shift(days=-1).format('YYYY-MM-DD')
    ldate = arrow.get(vdate).shift(years=-1).format('YYYY-MM-DD')
    vtime = str(train['시간'][i])
    if len(vtime) == 1: vtime = '0' + vtime
    vdates.append(vdate)
    tdates.append(tdate)
    ldates.append(ldate)
    vtimes.append(vtime)
    

 - API 요청 및 수집 코드

In [ ]:
datas_Hangang_mean = [] # 수계 기준으로 데이터를 수집할 경우 (여러 개의 댐 데이터를 평균)
datas_Chungju = []      # 댐 기준으로 데이터를 수집할 경우   (충주 댐만의 데이터)

# 반복문으로 공공데이터 포털 다목적댐 관리 현황 API 호출
for i in range(len(train)): # len(train) == 3051

    if i != 0 and i%500 == 0: print(f'데이터 {i}개 수집, 진행 중')
    url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/multipurPoseDam/list'
    queryParams = '?' + urlencode({
        quote_plus('ServiceKey') : ServiceKey,
        quote_plus('tdate') : tdates[i],
        quote_plus('ldate') : ldates[i],
        quote_plus('vdate') : vdates[i],
        quote_plus('vtime') : vtimes[i],
        quote_plus('numOfRows') : numOfRows,
        quote_plus('pageNo') : pageNo 
    })

    result = api_dataframe(url, queryParams)

    eng_columns = ['suge', 'damnm','zerosevenhourprcptqy','prcptqy','pyacurf','vyacurf','oyaacurf','inflowqy','totdcwtrqy','totdcwtrqyjo','nowlowlevel','lastlowlevel','nyearlowlevel','nowrsvwtqy','lastrsvwtqy','nyearrsvwtqy','rsvwtrt','dvlpqyacmtlacmslt','dvlpqyacmtlplan','dvlpqyacmtlversus','dvlpqyfyerplan','dvlpqyfyerversus']
    kor_columns = ['수계','댐이름','강우량 금일','강우량 전일','금년 누계강우량','전년 누계강우량','예년 누계강우량','전일 유입량','전일 방류량(본댐)','전일 방류량(조정지)','저수위(현재)','저수위(전년)','저수위(예년)','저수량(현재)','저수량(전년)','저수량(예년)','현재저수율','발전량(실적)','발전량(계획)','발전량(계획대비)','연간 발전계획','연간 계획대비']
    result = result[eng_columns]
    result = result.rename(columns=dict(zip(eng_columns, kor_columns)))


    # 데이터 수집01 : 수계가 한강인 경우
    data_hangang = result.query('수계 == "한강"')
    data_hangang_mean = data_hangang.applymap(lambda x : x.replace(',','')).iloc[:, 2:].astype(float).mean().agg(lambda x : x.round(3))
    datas_Hangang_mean.append(data_hangang_mean)


    # 데이터 수집02 : 댐이름이 충주댐인 경우
    data_chungju = result.query('댐이름 == "충주"')
    datas_Chungju.append(data_chungju)
    
print(f'모든 데이터 {i}개 수집 완료')


In [ ]:
# 수집한 데이터 저장
hangang_management = pd.concat(datas_Hangang_mean, axis=1).T
chungju_management = pd.concat(datas_Chungju, ignore_index=True)
hangang_management.to_csv('./data/다목적댐_관리현황(수계).csv')
chungju_management.to_csv('./data/다목적댐_관리현황(충주).csv')

In [55]:
# 데이터 확인
pd.read_csv('./data/다목적댐_관리현황(충주).csv')

,강우량 금일,강우량 전일,금년 누계강우량,전년 누계강우량,예년 누계강우량,전일 유입량,전일 방류량(본댐),전일 방류량(조정지),저수위(현재),저수위(전년),저수위(예년),저수량(현재),저수량(전년),저수량(예년),현재저수율,발전량(실적),발전량(계획),발전량(계획대비),연간 발전계획,연간 계획대비
0,0.4,4.7,488.0,585.7,475.2,228.8,402.7,469.5,122.61,128.92,121.46,"1,123.1","1,484.3","1,093.9",40.8,"266,851","323,452",82.5,"808,000",33.0
1,1.3,4.7,488.9,585.7,475.2,228.8,402.7,469.5,122.58,128.92,121.46,"1,121.5","1,484.3","1,093.9",40.8,"266,851","323,452",82.5,"808,000",33.0
2,2.2,4.7,489.8,585.7,475.2,228.8,402.7,469.5,122.54,128.92,121.46,"1,119.4","1,484.3","1,093.9",40.7,"266,851","323,452",82.5,"808,000",33.0
3,5.3,4.7,492.9,585.7,475.2,228.8,402.7,469.5,122.51,128.92,121.46,"1,117.9","1,484.3","1,093.9",40.7,"266,851","323,452",82.5,"808,000",33.0
4,8.4,4.7,496.0,585.7,475.2,228.8,402.7,469.5,122.49,128.92,121.46,"1,116.9","1,484.3","1,093.9",40.6,"266,851","323,452",82.5,"808,000",33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,0.4,1.4,691.1,421.0,456.4,979.3,197.1,320.4,129.91,127.10,122.22,"1,395.5","1,217.6",936.5,50.7,"311,510","227,888",136.7,"477,673",65.2
3047,0.4,1.4,691.1,421.0,456.4,979.3,197.1,320.4,129.92,127.10,122.22,"1,396.2","1,217.6",936.5,50.8,"311,510","227,888",136.7,"477,673",65.2
3048,0.4,1.4,691.1,421.0,456.4,979.3,197.1,320.4,129.91,127.10,122.22,"1,395.5","1,217.6",936.5,50.7,"311,510","227,888",136.7,"477,673",65.2
3049,0.4,1.4,691.1,421.0,456.4,979.3,197.1,320.4,129.89,127.10,122.22,"1,394.2","1,217.6",936.5,50.7,"311,510","227,888",136.7,"477,673",65.2


### 4. 수문 및 기타 데이터 수집
 - https://www.water.or.kr/realtime/sub01/sub01/dam/hydr.do?seq=1408&p_group_seq=1407&menu_mode=2 에서 연도별 엑셀파일들을 다운받아 사용하였음

 - 데이터 로드

In [65]:
data1 = pd.read_excel('./data/수문데이터연도별/excelData_2006.xlsx', header = 2)
data2 = pd.read_excel('./data/수문데이터연도별/excelData_2007_08.xlsx', header = 2)
data3 = pd.read_excel('./data/수문데이터연도별/excelData_2007_09.xlsx', header = 2)
data4 = pd.read_excel('./data/수문데이터연도별/excelData_2008.xlsx', header = 2)
data5 = pd.read_excel('./data/수문데이터연도별/excelData_2009_07.xlsx', header = 2)
data6 = pd.read_excel('./data/수문데이터연도별/excelData_2009_08.xlsx', header = 2)
data7 = pd.read_excel('./data/수문데이터연도별/excelData_2010_09.xlsx', header = 2)
data8 = pd.read_excel('./data/수문데이터연도별/excelData_2011_06.xlsx', header = 2)
data9 = pd.read_excel('./data/수문데이터연도별/excelData_2011_07.xlsx', header = 2)
data10 = pd.read_excel('./data/수문데이터연도별/excelData_2011_08.xlsx', header = 2)
data11 = pd.read_excel('./data/수문데이터연도별/excelData_2012_07.xlsx', header = 2)
data12 = pd.read_excel('./data/수문데이터연도별/excelData_2012_08.xlsx', header = 2)
data13 = pd.read_excel('./data/수문데이터연도별/excelData_2012_09.xlsx', header = 2)
data14 = pd.read_excel('./data/수문데이터연도별/excelData_2013_07.xlsx', header = 2)
data15 = pd.read_excel('./data/수문데이터연도별/excelData_2017_07.xlsx', header = 2)
data16 = pd.read_excel('./data/수문데이터연도별/excelData_2018_07.xlsx', header = 2)

 - train데이터에 붙이도록 데이터 전처리

In [66]:
# 기존 data의 연,월,일,시간 형식에 맞게 외부데이터의 형식을 재조정하기 위한 함수
def readjust_df(data):
    lst_mon = []
    lst_day = []
    lst_hour = []
    for i in data.일시:
        lst_mon.append(int(i[1:2]))
        lst_day.append(int(i[3:5]))
        lst_hour.append(int(i[6:8]))

    data['월'] = lst_mon
    data['일'] = lst_day
    data['시간'] = lst_hour

    del data['일시']
    return data


# 함수 적용
for i in range(1, 17):
    readjust_df(globals()[f'data{i}'])
data1['연'] = 2006
data2['연'] = 2007
data3['연'] = 2007
data4['연'] = 2008
data5['연'] = 2009
data6['연'] = 2009
data7['연'] = 2010
data8['연'] = 2011
data9['연'] = 2011
data10['연'] = 2011
data11['연'] = 2012
data12['연'] = 2012
data13['연'] = 2012
data14['연'] = 2013
data15['연'] = 2017
data16['연'] = 2018


# 위의 데이터들에서 병합하고자 하는 column만 뽑은 후 concat, 원본 데이터와 같은 3051rows로 병합
for i in range(1, 16):
    globals()[f'data{i}'] = globals()[f'data{i}'][['연', '월', '일', '시간', '댐수위(EL.m)', '방수로수위(EL.m)', '저수량(백만㎥)', '저수율(%)', '강우량(mm)', '유입량(㎥/s)',
          '자체유입(㎥/s)', '총방류량(㎥/s)', '발전(㎥/s)', '수문(㎥/s)', '하천유지(㎥/s)', '관개/하천유지(재오개)']]
lst_df = []
for i in range(1, 17):
    lst_df.append(globals()[f'data{i}'])

concat_df = pd.concat(lst_df, axis = 0)


# 원본데이터와 외부데이터 merge
merge_data = pd.merge(data, concat_df, on = ['연', '월', '일', '시간'])


# 최종 데이터를 위해 정리
opendoor_data = merge_data.loc[:, ['연', '월', '일', '시간', '댐수위(EL.m)', '방수로수위(EL.m)', '저수량(백만㎥)', '저수율(%)', '강우량(mm)', '유입량(㎥/s)',
          '자체유입(㎥/s)', '총방류량(㎥/s)', '발전(㎥/s)', '수문(㎥/s)', '하천유지(㎥/s)', '관개/하천유지(재오개)']]

opendoor_data.drop(['연', '월', '일', '시간', '저수량(백만㎥)', '저수율(%)', '발전(㎥/s)', '하천유지(㎥/s)', '관개/하천유지(재오개)', '유입량(㎥/s)'], 
                   axis = 1, inplace = True)
opendoor_data.rename(columns = {'댐수위(EL.m)' : '댐수위',
                      '방수로수위(EL.m)': '방수로수위',
                      '강우량(mm)': '강우량_해당시간',
                      '자체유입(㎥/s)': '자체유입',
                      '총방류량(㎥/s)': '총방류량',
                      '수문(㎥/s)': '수문'}, inplace = True)



# 최종 데이터 저장
opendoor_data.to_csv('./data/수문데이터 및 기타데이터.csv', encoding='cp949',index=False)
# 저장 데이터 확인
pd.read_csv('./data/수문데이터 및 기타데이터.csv', encoding='cp949')

,댐수위,방수로수위,강우량_해당시간,자체유입,총방류량,수문
0,122.61,66.74,0.4,143.889,0.00,0.000
1,122.58,69.94,0.9,129.754,560.87,0.000
2,122.54,70.03,0.9,97.138,671.58,0.000
3,122.51,70.13,3.1,268.040,698.04,0.000
4,122.49,70.13,3.1,416.401,703.07,0.000
...,...,...,...,...,...,...
3046,129.91,69.92,0.0,26.571,580.18,0.0
3047,129.92,69.92,0.0,757.916,573.47,0.0
3048,129.91,69.89,0.0,396.623,581.07,0.0
3049,129.89,70.00,0.0,228.407,597.3,0.0


### 5. 데이터 통합
 - 앞서 수집한 4가지의 데이터 확인 및 통합

In [134]:
### 수집한 4가지 종류의 데이터 로드
# 기상정보 데이터
weather_df = pd.read_csv('./data/기상데이터_최종.csv',encoding='cp949')
print('기상정보 데이터', end=' '); print(weather_df.shape)
display(weather_df.head(3))


# 수위관측소(수위, 유량) 데이터
observatory_df = pd.read_excel('./data/수위관측소(수위,유량).xlsx')
observatory_df.drop(['단양군(오사리)_수위'], axis=1, inplace=True) # 대회측 제공 데이터의 D, E와 동일
print('수위관측소 데이터', end=' '); print(observatory_df.shape)
display(observatory_df.head(3))


# 다목적댐 관리현황 데이터
multipurpose_dam_df = pd.read_csv('./data/다목적댐_관리현황(충주).csv') # 아래 칼럼들은 유입량과 관련이 없다 판단하여 제거
multipurpose_dam_df.drop(['발전량(실적)', '발전량(계획)', '발전량(계획대비)', '연간 발전계획', '연간 계획대비'], axis=1, inplace=True)
print('다목적댐 관리현황 데이터', end=' '); print(multipurpose_dam_df.shape)
display(multipurpose_dam_df.head(3))


# 수문 및 기타 데이터
floodgate_df = pd.read_csv('./data/수문데이터 및 기타데이터.csv', encoding='cp949')
print('수문 및 기타 데이터', end=' '); print(floodgate_df.shape)
display(floodgate_df.head(3))


# 데이터 통합
combinded_dateset = pd.concat([weather_df, observatory_df, multipurpose_dam_df, floodgate_df], axis=1)
print('통합 데이터', end=' '); print(combinded_dateset.shape)
display(combinded_dateset.head(3))

# 데이터 저장
combinded_dateset.to_csv('./data/수집데이터셋.csv', index=False)

기상정보 데이터 (3051, 7)


,기온,풍속,습도,증기압,이슬점온도,해면기압,지면온도
0,22.314286,2.200000,84.857143,23.078571,19.657143,1007.864286,12.700000
1,22.385714,2.200000,85.428571,23.500000,19.942857,1007.271429,22.242857
2,21.642857,2.871429,86.428571,23.500000,19.942857,1007.271429,12.700000


수위관측소 데이터 (3051, 10)


,단양군(오사리)_유량,영월군(북쌍리)_수위,영월군(북쌍리)_유량,영월군(판운교)_수위,영월군(판운교)_유량,우안(충주댐)_유량,제천시(청풍교)_유량,충주시(명서교)_유량,충주시(향산리)_수위,충주시(향산리)_유량
0,2.54,61.5,0.96,23.6,1.51,122.65,122.63,0.99,24.0,0.37
1,2.53,61.5,0.96,23.6,1.51,122.63,122.62,1.00,24.0,0.37
2,2.53,61.5,0.96,23.6,1.51,122.59,122.59,1.01,24.0,0.37


다목적댐 관리현황 데이터 (3051, 15)


,강우량 금일,강우량 전일,금년 누계강우량,전년 누계강우량,예년 누계강우량,전일 유입량,전일 방류량(본댐),전일 방류량(조정지),저수위(현재),저수위(전년),저수위(예년),저수량(현재),저수량(전년),저수량(예년),현재저수율
0,0.4,4.7,488.0,585.7,475.2,228.8,402.7,469.5,122.61,128.92,121.46,"1,123.1","1,484.3","1,093.9",40.8
1,1.3,4.7,488.9,585.7,475.2,228.8,402.7,469.5,122.58,128.92,121.46,"1,121.5","1,484.3","1,093.9",40.8
2,2.2,4.7,489.8,585.7,475.2,228.8,402.7,469.5,122.54,128.92,121.46,"1,119.4","1,484.3","1,093.9",40.7


수문 및 기타 데이터 (3051, 6)


,댐수위,방수로수위,강우량_해당시간,자체유입,총방류량,수문
0,122.61,66.74,0.4,143.889,0.00,0.000
1,122.58,69.94,0.9,129.754,560.87,0.000
2,122.54,70.03,0.9,97.138,671.58,0.000


통합 데이터 (3051, 38)


,기온,풍속,습도,증기압,이슬점온도,해면기압,지면온도,단양군(오사리)_유량,영월군(북쌍리)_수위,영월군(북쌍리)_유량,영월군(판운교)_수위,영월군(판운교)_유량,우안(충주댐)_유량,제천시(청풍교)_유량,충주시(명서교)_유량,...,전일 방류량(본댐),전일 방류량(조정지),저수위(현재),저수위(전년),저수위(예년),저수량(현재),저수량(전년),저수량(예년),현재저수율,댐수위,방수로수위,강우량_해당시간,자체유입,총방류량,수문
0,22.314286,2.200000,84.857143,23.078571,19.657143,1007.864286,12.700000,2.54,61.5,0.96,23.6,1.51,122.65,122.63,0.99,...,402.7,469.5,122.61,128.92,121.46,"1,123.1","1,484.3","1,093.9",40.8,122.61,66.74,0.4,143.889,0.00,0.000
1,22.385714,2.200000,85.428571,23.500000,19.942857,1007.271429,22.242857,2.53,61.5,0.96,23.6,1.51,122.63,122.62,1.00,...,402.7,469.5,122.58,128.92,121.46,"1,121.5","1,484.3","1,093.9",40.8,122.58,69.94,0.9,129.754,560.87,0.000
2,21.642857,2.871429,86.428571,23.500000,19.942857,1007.271429,12.700000,2.53,61.5,0.96,23.6,1.51,122.59,122.59,1.01,...,402.7,469.5,122.54,128.92,121.46,"1,119.4","1,484.3","1,093.9",40.7,122.54,70.03,0.9,97.138,671.58,0.000


# ───────────────── End of Pipeline 1/4  ─────────────────